In [1]:
import torch
import numpy as np
import load_data

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torchvision import datasets, transforms

from se3cnn import SE3Convolution, SE3Dropout
from se3cnn.blocks import GatedBlock
from se3cnn.non_linearities import ScalarActivation
from se3cnn.dropout import SE3Dropout
from se3cnn import kernel
from se3cnn.filter import low_pass_filter

from tensorflow.python.framework import dtypes

In [2]:
def rotation_1(t):
    return torch.flip(t, (3, )).transpose(4, 3)

def untransformed(n, size):
    untransf = []
    for i in range(n):
        a_i = torch.zeros(1,1,size,size,size)
        for j in range(size):
            a_i[0,0,j,j,0] = np.random.randn()
            a_i[0,0,j,0,j] = a_i[0,0,j,j,0]
            a_i[0,0,0,j,j] = a_i[0,0,j,j,0]
            
        untransf.extend([(a_i, torch.tensor([1,1,1], dtype=torch.float32)), 
                         (rotation_1(a_i), torch.tensor([-1,1,1], dtype=torch.float32)), 
                         (rotation_1(rotation_1(a_i)), torch.tensor([-1,1,-1], dtype=torch.float32)), 
                         (rotation_1(rotation_1(rotation_1(a_i))), torch.tensor([1,1,-1], dtype=torch.float32))
                        ])
    return untransf

In [140]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        #self.gated = PoolGatedBlock(repr_in, repr_out, size, activation=activation, 
                                    #pool_size=pool_size, pool_stride=pool_stride, bias=bias)
        # self.conv = nn.Conv3d(1, 2, 2, 2, 1)
        self.conv1 = SE3Convolution(repr_in_1, repr_out_2, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.drop1 = nn.Dropout(prob)
        self.lin2 = nn.Linear(n_output_1, n_output_2)
        self.drop2 = nn.Dropout(prob)
        self.lin3 = nn.Linear(n_output_2, NUM_CLASSES)

    def forward(self, x):
        # x = self.gated(x)
        x = self.pool1(self.conv1(x))
        #x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) # tf.reshape(x,[batchSize,-1])
        x = F.leaky_relu(self.lin1(x))
        # x = self.drop1(x)
        # x = F.relu(self.lin2(x))
        # x = self.drop2(x)
        return self.lin2(x) #self.lin3(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    print(train_set._index_in_epoch)
    print(train_set._num_examples)
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        loss_fn = nn.MSELoss(reduction='sum')
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print(lr)
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        loss_fn = nn.MSELoss(reduction='sum')
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            # data, target = data.to(device), target.to(device)
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) # sum up batch loss
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [141]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2)]
repr_in_2 = [(2,0),(2,1),(2,2)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 7000
n_output_1 = 3500 
n_output_2 = 7 

batch_size = 100
prob = 0.5
NUM_CLASSES = 1

In [142]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [143]:
epochs = 1000
device = torch.device('cpu')
torch.manual_seed(1)

# matrix_size = 16
# train_set = untransformed(50, matrix_size)
'''
test_set = [(a_i, torch.tensor([1,1,1], dtype=torch.float32)), 
            (rotation_1(a_i), torch.tensor([-1,1,1], dtype=torch.float32)), 
            (rotation_1(rotation_1(a_i)), torch.tensor([-1,1,-1], dtype=torch.float32)), 
            (rotation_1(rotation_1(rotation_1(a_i))), torch.tensor([1,1,-1], dtype=torch.float32))]
'''

model_hard = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_hard.parameters(), lr=learning_rate)

per_epoch = 150
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_hard, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

0
39785
Train Epoch: 1 [1000/39785 (0%)]	Loss: 2721.561279
Train Epoch: 1 [2000/39785 (0%)]	Loss: 1074.573975
Train Epoch: 1 [3000/39785 (0%)]	Loss: 973.752808
Train Epoch: 1 [4000/39785 (0%)]	Loss: 829.442627
Train Epoch: 1 [5000/39785 (0%)]	Loss: 574.667908
Train Epoch: 1 [6000/39785 (0%)]	Loss: 739.444763
Train Epoch: 1 [7000/39785 (0%)]	Loss: 558.614746
Train Epoch: 1 [8000/39785 (0%)]	Loss: 556.047913
Train Epoch: 1 [9000/39785 (0%)]	Loss: 472.887207
Train Epoch: 1 [10000/39785 (0%)]	Loss: 657.138245
Train Epoch: 1 [11000/39785 (0%)]	Loss: 715.385254
Train Epoch: 1 [12000/39785 (0%)]	Loss: 561.910278
Train Epoch: 1 [13000/39785 (0%)]	Loss: 463.862244
Train Epoch: 1 [14000/39785 (0%)]	Loss: 558.785278
Train Epoch: 1 [15000/39785 (0%)]	Loss: 489.248169
Train Epoch: 1 [16000/39785 (0%)]	Loss: 550.416016
Train Epoch: 1 [17000/39785 (0%)]	Loss: 524.253906
Train Epoch: 1 [18000/39785 (0%)]	Loss: 577.391418
Train Epoch: 1 [19000/39785 (0%)]	Loss: 370.317200
Train Epoch: 1 [20000/39785 (1

Train Epoch: 5 [6000/39785 (0%)]	Loss: 267.314728
Train Epoch: 5 [7000/39785 (0%)]	Loss: 250.307434
Train Epoch: 5 [8000/39785 (0%)]	Loss: 281.507568
Train Epoch: 5 [9000/39785 (0%)]	Loss: 182.773804
Train Epoch: 5 [10000/39785 (0%)]	Loss: 257.191284
Train Epoch: 5 [11000/39785 (0%)]	Loss: 200.231445
Train Epoch: 5 [12000/39785 (0%)]	Loss: 229.285095
Train Epoch: 5 [13000/39785 (0%)]	Loss: 229.772919
Train Epoch: 5 [14000/39785 (0%)]	Loss: 212.128021
Train Epoch: 5 [15000/39785 (0%)]	Loss: 199.787247
Train Epoch: 5 [16000/39785 (0%)]	Loss: 271.689301
Train Epoch: 5 [17000/39785 (0%)]	Loss: 221.210098
Train Epoch: 5 [18000/39785 (0%)]	Loss: 274.165039
Train Epoch: 5 [19000/39785 (0%)]	Loss: 268.127228
Train Epoch: 5 [20000/39785 (1%)]	Loss: 301.598602
Train Epoch: 5 [21000/39785 (1%)]	Loss: 254.278824
Train Epoch: 5 [22000/39785 (1%)]	Loss: 271.156189
Train Epoch: 5 [23000/39785 (1%)]	Loss: 238.583755
Train Epoch: 5 [24000/39785 (1%)]	Loss: 270.412384
Train Epoch: 5 [25000/39785 (1%)]	L

Train Epoch: 9 [11000/39785 (0%)]	Loss: 159.807724
Train Epoch: 9 [12000/39785 (0%)]	Loss: 203.979172
Train Epoch: 9 [13000/39785 (0%)]	Loss: 256.048859
Train Epoch: 9 [14000/39785 (0%)]	Loss: 234.559647
Train Epoch: 9 [15000/39785 (0%)]	Loss: 132.976486
Train Epoch: 9 [16000/39785 (0%)]	Loss: 133.622467
Train Epoch: 9 [17000/39785 (0%)]	Loss: 250.525925
Train Epoch: 9 [18000/39785 (0%)]	Loss: 147.256393
Train Epoch: 9 [19000/39785 (0%)]	Loss: 201.704956
Train Epoch: 9 [20000/39785 (1%)]	Loss: 182.743790
Train Epoch: 9 [21000/39785 (1%)]	Loss: 169.760818
Train Epoch: 9 [22000/39785 (1%)]	Loss: 213.394302
Train Epoch: 9 [23000/39785 (1%)]	Loss: 196.216064
Train Epoch: 9 [24000/39785 (1%)]	Loss: 182.808990
Train Epoch: 9 [25000/39785 (1%)]	Loss: 180.911133
Train Epoch: 9 [26000/39785 (1%)]	Loss: 185.825043
Train Epoch: 9 [27000/39785 (1%)]	Loss: 224.164673
Train Epoch: 9 [28000/39785 (1%)]	Loss: 137.947769
Train Epoch: 9 [29000/39785 (1%)]	Loss: 258.190155
Train Epoch: 9 [30000/39785 (1%

Train Epoch: 13 [13000/39785 (0%)]	Loss: 106.710693
Train Epoch: 13 [14000/39785 (0%)]	Loss: 134.318665
Train Epoch: 13 [15000/39785 (0%)]	Loss: 139.144791
Train Epoch: 13 [16000/39785 (0%)]	Loss: 133.970505
Train Epoch: 13 [17000/39785 (0%)]	Loss: 160.414810
Train Epoch: 13 [18000/39785 (0%)]	Loss: 145.912033
Train Epoch: 13 [19000/39785 (0%)]	Loss: 104.092735
Train Epoch: 13 [20000/39785 (1%)]	Loss: 200.826721
Train Epoch: 13 [21000/39785 (1%)]	Loss: 155.518921
Train Epoch: 13 [22000/39785 (1%)]	Loss: 137.370850
Train Epoch: 13 [23000/39785 (1%)]	Loss: 159.411438
Train Epoch: 13 [24000/39785 (1%)]	Loss: 1535.654663
Train Epoch: 13 [25000/39785 (1%)]	Loss: 1012.830933
Train Epoch: 13 [26000/39785 (1%)]	Loss: 5798.372070
Train Epoch: 13 [27000/39785 (1%)]	Loss: 750.386230
Train Epoch: 13 [28000/39785 (1%)]	Loss: 436.498260
Train Epoch: 13 [29000/39785 (1%)]	Loss: 425.761597
Train Epoch: 13 [30000/39785 (1%)]	Loss: 410.461060
Train Epoch: 13 [31000/39785 (1%)]	Loss: 365.869720
Train Epo

Train Epoch: 17 [15000/39785 (0%)]	Loss: 305.975708
Train Epoch: 17 [16000/39785 (0%)]	Loss: 338.773560
Train Epoch: 17 [17000/39785 (0%)]	Loss: 365.518311
Train Epoch: 17 [18000/39785 (0%)]	Loss: 254.789993
Train Epoch: 17 [19000/39785 (0%)]	Loss: 391.823395
Train Epoch: 17 [20000/39785 (1%)]	Loss: 361.137756
Train Epoch: 17 [21000/39785 (1%)]	Loss: 238.595108
Train Epoch: 17 [22000/39785 (1%)]	Loss: 311.331635
Train Epoch: 17 [23000/39785 (1%)]	Loss: 269.847229
Train Epoch: 17 [24000/39785 (1%)]	Loss: 256.381744
Train Epoch: 17 [25000/39785 (1%)]	Loss: 238.524673
Train Epoch: 17 [26000/39785 (1%)]	Loss: 196.899612
Train Epoch: 17 [27000/39785 (1%)]	Loss: 305.801971
Train Epoch: 17 [28000/39785 (1%)]	Loss: 268.996948
Train Epoch: 17 [29000/39785 (1%)]	Loss: 13286.778320
Train Epoch: 17 [30000/39785 (1%)]	Loss: 1313.130127
Train Epoch: 17 [31000/39785 (1%)]	Loss: 1179.322876
Train Epoch: 17 [32000/39785 (1%)]	Loss: 782.432617
Train Epoch: 17 [33000/39785 (1%)]	Loss: 605.575012
Train Ep

...

Train Epoch: 43 [31000/39785 (1%)]	Loss: 55.441082
Train Epoch: 43 [32000/39785 (1%)]	Loss: 70.763359
Train Epoch: 43 [33000/39785 (1%)]	Loss: 83.316055
Train Epoch: 43 [34000/39785 (1%)]	Loss: 50.247536
Train Epoch: 43 [35000/39785 (1%)]	Loss: 51.767487
Train Epoch: 43 [36000/39785 (1%)]	Loss: 62.485100
Train Epoch: 43 [37000/39785 (1%)]	Loss: 44.350361
Train Epoch: 43 [38000/39785 (1%)]	Loss: 54.388588
Train Epoch: 43 [39000/39785 (1%)]	Loss: 68.870529
39700
39785
Train Epoch: 44 [1000/39785 (0%)]	Loss: 67.387611
Train Epoch: 44 [2000/39785 (0%)]	Loss: 54.107941
Train Epoch: 44 [3000/39785 (0%)]	Loss: 48.686325
Train Epoch: 44 [4000/39785 (0%)]	Loss: 49.457703
Train Epoch: 44 [5000/39785 (0%)]	Loss: 43.195225
Train Epoch: 44 [6000/39785 (0%)]	Loss: 59.877464
Train Epoch: 44 [7000/39785 (0%)]	Loss: 48.908813
Train Epoch: 44 [8000/39785 (0%)]	Loss: 54.660458
Train Epoch: 44 [9000/39785 (0%)]	Loss: 45.513191
Train Epoch: 44 [10000/39785 (0%)]	Loss: 60.584431
Train Epoch: 44 [11000/39785

Train Epoch: 47 [36000/39785 (1%)]	Loss: 40.898792
Train Epoch: 47 [37000/39785 (1%)]	Loss: 45.971962
Train Epoch: 47 [38000/39785 (1%)]	Loss: 34.702728
Train Epoch: 47 [39000/39785 (1%)]	Loss: 42.223938
39760
39785
Train Epoch: 48 [1000/39785 (0%)]	Loss: 43.219185
Train Epoch: 48 [2000/39785 (0%)]	Loss: 34.502762
Train Epoch: 48 [3000/39785 (0%)]	Loss: 41.055058
Train Epoch: 48 [4000/39785 (0%)]	Loss: 31.658493
Train Epoch: 48 [5000/39785 (0%)]	Loss: 39.151028
Train Epoch: 48 [6000/39785 (0%)]	Loss: 229.345078
Train Epoch: 48 [7000/39785 (0%)]	Loss: 40.306435
Train Epoch: 48 [8000/39785 (0%)]	Loss: 37.408676
Train Epoch: 48 [9000/39785 (0%)]	Loss: 33.568874
Train Epoch: 48 [10000/39785 (0%)]	Loss: 33.257336
Train Epoch: 48 [11000/39785 (0%)]	Loss: 61.300510
Train Epoch: 48 [12000/39785 (0%)]	Loss: 41.643398
Train Epoch: 48 [13000/39785 (0%)]	Loss: 45.624577
Train Epoch: 48 [14000/39785 (0%)]	Loss: 42.983139
Train Epoch: 48 [15000/39785 (0%)]	Loss: 29.735014
Train Epoch: 48 [16000/3978

Train Epoch: 52 [1000/39785 (0%)]	Loss: 38.039146
Train Epoch: 52 [2000/39785 (0%)]	Loss: 31.030167
Train Epoch: 52 [3000/39785 (0%)]	Loss: 39.226330
Train Epoch: 52 [4000/39785 (0%)]	Loss: 36.567795
Train Epoch: 52 [5000/39785 (0%)]	Loss: 33.601250
Train Epoch: 52 [6000/39785 (0%)]	Loss: 28.641104
Train Epoch: 52 [7000/39785 (0%)]	Loss: 27.337687
Train Epoch: 52 [8000/39785 (0%)]	Loss: 43.180580
Train Epoch: 52 [9000/39785 (0%)]	Loss: 45.932602
Train Epoch: 52 [10000/39785 (0%)]	Loss: 40.116055
Train Epoch: 52 [11000/39785 (0%)]	Loss: 36.361622
Train Epoch: 52 [12000/39785 (0%)]	Loss: 38.304241
Train Epoch: 52 [13000/39785 (0%)]	Loss: 29.187811
Train Epoch: 52 [14000/39785 (0%)]	Loss: 33.373589
Train Epoch: 52 [15000/39785 (0%)]	Loss: 27.518009
Train Epoch: 52 [16000/39785 (0%)]	Loss: 33.967640
Train Epoch: 52 [17000/39785 (0%)]	Loss: 41.698776
Train Epoch: 52 [18000/39785 (0%)]	Loss: 47.840401
Train Epoch: 52 [19000/39785 (0%)]	Loss: 43.202614
Train Epoch: 52 [20000/39785 (1%)]	Loss:

Train Epoch: 56 [6000/39785 (0%)]	Loss: 29.782087
Train Epoch: 56 [7000/39785 (0%)]	Loss: 24.443213
Train Epoch: 56 [8000/39785 (0%)]	Loss: 22.717445
Train Epoch: 56 [9000/39785 (0%)]	Loss: 25.146513
Train Epoch: 56 [10000/39785 (0%)]	Loss: 23.430019
Train Epoch: 56 [11000/39785 (0%)]	Loss: 25.108603
Train Epoch: 56 [12000/39785 (0%)]	Loss: 348.366119
Train Epoch: 56 [13000/39785 (0%)]	Loss: 25.848816
Train Epoch: 56 [14000/39785 (0%)]	Loss: 18.311773
Train Epoch: 56 [15000/39785 (0%)]	Loss: 24.141392
Train Epoch: 56 [16000/39785 (0%)]	Loss: 27.882996
Train Epoch: 56 [17000/39785 (0%)]	Loss: 25.426882
Train Epoch: 56 [18000/39785 (0%)]	Loss: 21.784637
Train Epoch: 56 [19000/39785 (0%)]	Loss: 23.197939
Train Epoch: 56 [20000/39785 (1%)]	Loss: 18.033220
Train Epoch: 56 [21000/39785 (1%)]	Loss: 19.812561
Train Epoch: 56 [22000/39785 (1%)]	Loss: 22.534891
Train Epoch: 56 [23000/39785 (1%)]	Loss: 21.964024
Train Epoch: 56 [24000/39785 (1%)]	Loss: 18.788759
Train Epoch: 56 [25000/39785 (1%)]

KeyboardInterrupt: 

In [148]:
answer = model_hard(torch.from_numpy(train_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [149]:
answer[1]

tensor([ 0.4324,  0.8455,  0.0822,  0.1313, -0.1046, -0.4683,  0.2780],
       grad_fn=<SelectBackward>)

In [150]:
train_set._labels[1]

array([ 1.        ,  0.86292964,  0.03844497,  0.09862536, -0.08708213,
       -0.41256937,  0.25758612], dtype=float32)

In [142]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [151]:
test_name = '../deepSymmetry/data/densityMaps/density_testf'
test_set = load_data.read_data_set(test_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/densityMaps/density_testf
Extracting ../deepSymmetry/data/densityMaps/density_testf_label
(120000, 13824)


In [157]:
answer_test = model_hard(torch.from_numpy(test_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [179]:
for i in range(10):
    print(answer_test[i].data.tolist())
    print(test_set._labels[i])
    print()

[6.681089878082275, 0.8225629329681396, 0.24174951016902924, 0.033087629824876785, 0.010919880121946335, -0.17829450964927673, 0.6451407074928284]
[ 7.          0.8140826   0.16944312  0.01647427 -0.07471883 -0.16377677
  0.5252441 ]

[7.156384468078613, -0.09772703796625137, 0.8188420534133911, 0.08037760853767395, -0.6574083566665649, 0.03126903250813484, -0.26504260301589966]
[ 7.          0.01401387  0.8029993   0.18298681 -0.5421038   0.071615
 -0.15002087]

[8.394238471984863, 1.0517383813858032, 0.08680028468370438, 0.1367802619934082, 0.03753487020730972, 0.0565904825925827, -0.18338654935359955]
[ 7.0000000e+00  9.5146030e-01  4.8440255e-02  9.9439159e-05
 -3.1038227e-03  1.3755901e-02 -3.0360824e-01]

[7.138195991516113, 0.21090511977672577, 0.5391544103622437, 0.294271856546402, -0.5449842214584351, -0.480374276638031, 0.7109291553497314]
[ 7.          0.23470888  0.4653611   0.29993004 -0.5283479  -0.37522325
  0.46738502]

[8.730587005615234, 0.6358888149261475, 0.33428075

---

In [225]:
def custom_loss(output, target):
    order_out = output[:, 0 : NUM_CLASSES]
    order_target = target[:, 0 : 1].type(torch.LongTensor).squeeze_()
    axis_out = output[:, NUM_CLASSES : NUM_CLASSES + 6]
    axis_target = target[:, 1 : 7]

    loss = nn.CrossEntropyLoss()(order_out, order_target) + nn.MSELoss(reduction='sum')(axis_out, axis_target)
    
    return loss

In [226]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SE3Convolution(repr_in_1, repr_out_1, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.drop1 = nn.Dropout(prob)
        self.lin2 = nn.Linear(n_output_1, n_output_2)
        self.drop2 = nn.Dropout(prob)
        self.lin3 = nn.Linear(n_output_2, NUM_CLASSES)

    def forward(self, x):
        # x = self.gated(x)
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) # tf.reshape(x,[batchSize,-1])
        x = F.leaky_relu(self.lin1(x))
        # x = self.drop1(x)
        # x = F.relu(self.lin2(x))
        # x = self.drop2(x)
        return self.lin2(x) #self.lin3(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        else:
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) 
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [227]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2),(2,3)]
repr_in_2 = [(2,0),(2,1),(2,2),(2,3)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 189
n_output_1 = 56 
n_output_2 = NUM_CLASSES + 6 

batch_size = 100
prob = 0.5
NUM_CLASSES = 10

In [228]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [229]:
epochs = 100
device = torch.device('cpu')
torch.manual_seed(1)

model_custom_loss = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_custom_loss.parameters(), lr=learning_rate)

per_epoch = 50
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_custom_loss, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

Train Epoch: 1 [1000/39785 (0%)]	Loss: 46.648594
Train Epoch: 1 [2000/39785 (0%)]	Loss: 41.696766
Train Epoch: 1 [3000/39785 (0%)]	Loss: 35.723019
Train Epoch: 1 [4000/39785 (0%)]	Loss: 32.567127
Train Epoch: 1 [5000/39785 (0%)]	Loss: 26.332949
Train Epoch: 1 [6000/39785 (0%)]	Loss: 33.494045
Train Epoch: 1 [7000/39785 (0%)]	Loss: 27.060057
Train Epoch: 1 [8000/39785 (0%)]	Loss: 19.113297
Train Epoch: 1 [9000/39785 (0%)]	Loss: 22.934341
Train Epoch: 1 [10000/39785 (0%)]	Loss: 24.767778
Train Epoch: 1 [11000/39785 (0%)]	Loss: 24.873268
Train Epoch: 1 [12000/39785 (0%)]	Loss: 21.899326
Train Epoch: 1 [13000/39785 (0%)]	Loss: 20.926369
Train Epoch: 1 [14000/39785 (0%)]	Loss: 22.999231
Train Epoch: 1 [15000/39785 (0%)]	Loss: 20.996355
Train Epoch: 1 [16000/39785 (0%)]	Loss: 21.345695
Train Epoch: 1 [17000/39785 (0%)]	Loss: 24.683270
Train Epoch: 1 [18000/39785 (0%)]	Loss: 19.233530
Train Epoch: 1 [19000/39785 (0%)]	Loss: 22.725492
Train Epoch: 1 [20000/39785 (1%)]	Loss: 22.082321
Train Epo

Train Epoch: 5 [10000/39785 (0%)]	Loss: 12.622517
Train Epoch: 5 [11000/39785 (0%)]	Loss: 14.746858
Train Epoch: 5 [12000/39785 (0%)]	Loss: 10.616294
Train Epoch: 5 [13000/39785 (0%)]	Loss: 17.223125
Train Epoch: 5 [14000/39785 (0%)]	Loss: 15.697552
Train Epoch: 5 [15000/39785 (0%)]	Loss: 12.174756
Train Epoch: 5 [16000/39785 (0%)]	Loss: 14.795235
Train Epoch: 5 [17000/39785 (0%)]	Loss: 17.448853
Train Epoch: 5 [18000/39785 (0%)]	Loss: 15.579608
Train Epoch: 5 [19000/39785 (0%)]	Loss: 14.769566
Train Epoch: 5 [20000/39785 (1%)]	Loss: 11.806815
Train Epoch: 5 [21000/39785 (1%)]	Loss: 12.723416
Train Epoch: 5 [22000/39785 (1%)]	Loss: 13.099526
Train Epoch: 5 [23000/39785 (1%)]	Loss: 14.445068
Train Epoch: 5 [24000/39785 (1%)]	Loss: 13.790761
Train Epoch: 5 [25000/39785 (1%)]	Loss: 14.738664
Train Epoch: 5 [26000/39785 (1%)]	Loss: 14.207312
Train Epoch: 5 [27000/39785 (1%)]	Loss: 13.244943
Train Epoch: 5 [28000/39785 (1%)]	Loss: 15.262305
Train Epoch: 5 [29000/39785 (1%)]	Loss: 15.167724


Train Epoch: 9 [19000/39785 (0%)]	Loss: 18.231171
Train Epoch: 9 [20000/39785 (1%)]	Loss: 14.949144
Train Epoch: 9 [21000/39785 (1%)]	Loss: 12.592390
Train Epoch: 9 [22000/39785 (1%)]	Loss: 12.259623
Train Epoch: 9 [23000/39785 (1%)]	Loss: 14.120815
Train Epoch: 9 [24000/39785 (1%)]	Loss: 15.186064
Train Epoch: 9 [25000/39785 (1%)]	Loss: 16.055338
Train Epoch: 9 [26000/39785 (1%)]	Loss: 16.827185
Train Epoch: 9 [27000/39785 (1%)]	Loss: 13.573219
Train Epoch: 9 [28000/39785 (1%)]	Loss: 16.488340
Train Epoch: 9 [29000/39785 (1%)]	Loss: 15.778923
Train Epoch: 9 [30000/39785 (1%)]	Loss: 13.095004
Train Epoch: 9 [31000/39785 (1%)]	Loss: 15.102017
Train Epoch: 9 [32000/39785 (1%)]	Loss: 10.513493
Train Epoch: 9 [33000/39785 (1%)]	Loss: 14.453828
Train Epoch: 9 [34000/39785 (1%)]	Loss: 14.549540
Train Epoch: 9 [35000/39785 (1%)]	Loss: 10.506563
Train Epoch: 9 [36000/39785 (1%)]	Loss: 14.719914
Train Epoch: 9 [37000/39785 (1%)]	Loss: 14.500222
Train Epoch: 9 [38000/39785 (1%)]	Loss: 17.233698


KeyboardInterrupt: 

In [230]:
answer_test = model_custom_loss(torch.from_numpy(test_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [240]:
for i in range(10):
    print(torch.argmax(answer_test[i][:NUM_CLASSES]), answer_test[i][NUM_CLASSES:].data.tolist())
    print(test_set._labels[i])
    print()

tensor(7) [0.9126054644584656, 0.13815365731716156, 0.0009435713291168213, -0.14054299890995026, -0.22608059644699097, 0.6172221302986145]
[ 7.          0.8140826   0.16944312  0.01647427 -0.07471883 -0.16377677
  0.5252441 ]

tensor(8) [0.045435696840286255, 0.9039760828018188, 0.22042083740234375, -0.5384350419044495, -0.03082834556698799, -0.058054883033037186]
[ 7.          0.01401387  0.8029993   0.18298681 -0.5421038   0.071615
 -0.15002087]

tensor(9) [1.028212308883667, 0.037577927112579346, -0.004926323890686035, -0.13047687709331512, 0.028444167226552963, -0.27478092908859253]
[ 7.0000000e+00  9.5146030e-01  4.8440255e-02  9.9439159e-05
 -3.1038227e-03  1.3755901e-02 -3.0360824e-01]

tensor(9) [0.20076383650302887, 0.5157843828201294, 0.2729910612106323, -0.6000676155090332, -0.36886218190193176, 0.5529150366783142]
[ 7.          0.23470888  0.4653611   0.29993004 -0.5283479  -0.37522325
  0.46738502]

tensor(7) [0.5473963022232056, 0.37036943435668945, 0.22840455174446106, -

---

In [258]:
def weighted_custom_loss(output, target):
    order_out = output[:, 0 : NUM_CLASSES]
    order_target = target[:, 0 : 1].type(torch.LongTensor).squeeze_()
    axis_out = output[:, NUM_CLASSES : NUM_CLASSES + 6]
    axis_target = target[:, 1 : 7]

    loss = 1 * nn.CrossEntropyLoss()(order_out, order_target) + nn.MSELoss(reduction='sum')(axis_out, axis_target)
    
    return loss

In [260]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SE3Convolution(repr_in_1, repr_out_1, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.drop1 = nn.Dropout(prob)
        self.lin2 = nn.Linear(n_output_1, n_output_2)
        self.drop2 = nn.Dropout(prob)
        self.lin3 = nn.Linear(n_output_2, NUM_CLASSES+6)

    def forward(self, x):
        # x = self.gated(x)
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) # tf.reshape(x,[batchSize,-1])
        x = F.leaky_relu(self.lin1(x))
        x = self.drop1(x)
        x = F.leaky_relu(self.lin2(x))
        x = self.drop2(x)
        return self.lin3(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = weighted_custom_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) 
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [261]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2),(2,3)]
repr_in_2 = [(2,0),(2,1),(2,2),(2,3)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 189
n_output_1 = 74 
n_output_2 = 40

batch_size = 100
prob = 0.5
NUM_CLASSES = 10

In [262]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [263]:
epochs = 100
device = torch.device('cpu')
torch.manual_seed(1)

model_custom_loss = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_custom_loss.parameters(), lr=learning_rate)

per_epoch = 5
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_custom_loss, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

Train Epoch: 1 [1000/39785 (0%)]	Loss: 67.335251
Train Epoch: 1 [2000/39785 (0%)]	Loss: 58.905796
Train Epoch: 1 [3000/39785 (0%)]	Loss: 56.228504
Train Epoch: 1 [4000/39785 (0%)]	Loss: 50.652225
Train Epoch: 1 [5000/39785 (0%)]	Loss: 51.507225
Train Epoch: 1 [6000/39785 (0%)]	Loss: 50.058414
Train Epoch: 1 [7000/39785 (0%)]	Loss: 49.381378
Train Epoch: 1 [8000/39785 (0%)]	Loss: 41.233288
Train Epoch: 1 [9000/39785 (0%)]	Loss: 45.783123
Train Epoch: 1 [10000/39785 (0%)]	Loss: 42.588291
Train Epoch: 1 [11000/39785 (0%)]	Loss: 39.424538
Train Epoch: 1 [12000/39785 (0%)]	Loss: 40.566345
Train Epoch: 1 [13000/39785 (0%)]	Loss: 39.995770
Train Epoch: 1 [14000/39785 (0%)]	Loss: 37.852776
Train Epoch: 1 [15000/39785 (0%)]	Loss: 36.517467
Train Epoch: 1 [16000/39785 (0%)]	Loss: 39.119663
Train Epoch: 1 [17000/39785 (0%)]	Loss: 39.379356
Train Epoch: 1 [18000/39785 (0%)]	Loss: 32.916790
Train Epoch: 1 [19000/39785 (0%)]	Loss: 40.311321
Train Epoch: 1 [20000/39785 (1%)]	Loss: 39.725632
Train Epo

Train Epoch: 5 [10000/39785 (0%)]	Loss: 27.056166
Train Epoch: 5 [11000/39785 (0%)]	Loss: 25.496437
Train Epoch: 5 [12000/39785 (0%)]	Loss: 21.688648
Train Epoch: 5 [13000/39785 (0%)]	Loss: 28.641411
Train Epoch: 5 [14000/39785 (0%)]	Loss: 29.990101
Train Epoch: 5 [15000/39785 (0%)]	Loss: 24.634541
Train Epoch: 5 [16000/39785 (0%)]	Loss: 27.037546
Train Epoch: 5 [17000/39785 (0%)]	Loss: 29.249025
Train Epoch: 5 [18000/39785 (0%)]	Loss: 27.435038
Train Epoch: 5 [19000/39785 (0%)]	Loss: 27.575462
Train Epoch: 5 [20000/39785 (1%)]	Loss: 23.076130
Train Epoch: 5 [21000/39785 (1%)]	Loss: 25.909668
Train Epoch: 5 [22000/39785 (1%)]	Loss: 25.048637
Train Epoch: 5 [23000/39785 (1%)]	Loss: 27.880983
Train Epoch: 5 [24000/39785 (1%)]	Loss: 30.983496
Train Epoch: 5 [25000/39785 (1%)]	Loss: 24.562962
Train Epoch: 5 [26000/39785 (1%)]	Loss: 29.923880
Train Epoch: 5 [27000/39785 (1%)]	Loss: 26.116508
Train Epoch: 5 [28000/39785 (1%)]	Loss: 31.898441
Train Epoch: 5 [29000/39785 (1%)]	Loss: 27.364183


Train Epoch: 9 [19000/39785 (0%)]	Loss: 29.666615
Train Epoch: 9 [20000/39785 (1%)]	Loss: 25.991741
Train Epoch: 9 [21000/39785 (1%)]	Loss: 26.139845
Train Epoch: 9 [22000/39785 (1%)]	Loss: 25.967220
Train Epoch: 9 [23000/39785 (1%)]	Loss: 25.522402
Train Epoch: 9 [24000/39785 (1%)]	Loss: 30.264156
Train Epoch: 9 [25000/39785 (1%)]	Loss: 25.929882
Train Epoch: 9 [26000/39785 (1%)]	Loss: 25.565189
Train Epoch: 9 [27000/39785 (1%)]	Loss: 27.156290
Train Epoch: 9 [28000/39785 (1%)]	Loss: 28.622025
Train Epoch: 9 [29000/39785 (1%)]	Loss: 25.866623
Train Epoch: 9 [30000/39785 (1%)]	Loss: 25.579834
Train Epoch: 9 [31000/39785 (1%)]	Loss: 26.742338
Train Epoch: 9 [32000/39785 (1%)]	Loss: 21.939323
Train Epoch: 9 [33000/39785 (1%)]	Loss: 28.412472
Train Epoch: 9 [34000/39785 (1%)]	Loss: 27.044867
Train Epoch: 9 [35000/39785 (1%)]	Loss: 22.056583
Train Epoch: 9 [36000/39785 (1%)]	Loss: 28.542240
Train Epoch: 9 [37000/39785 (1%)]	Loss: 28.105656
Train Epoch: 9 [38000/39785 (1%)]	Loss: 28.666237


Train Epoch: 13 [25000/39785 (1%)]	Loss: 27.450760
Train Epoch: 13 [26000/39785 (1%)]	Loss: 23.736397
Train Epoch: 13 [27000/39785 (1%)]	Loss: 26.725424
Train Epoch: 13 [28000/39785 (1%)]	Loss: 26.009037
Train Epoch: 13 [29000/39785 (1%)]	Loss: 26.271196
Train Epoch: 13 [30000/39785 (1%)]	Loss: 25.008944
Train Epoch: 13 [31000/39785 (1%)]	Loss: 26.122383
Train Epoch: 13 [32000/39785 (1%)]	Loss: 23.952736
Train Epoch: 13 [33000/39785 (1%)]	Loss: 24.518227
Train Epoch: 13 [34000/39785 (1%)]	Loss: 27.931667
Train Epoch: 13 [35000/39785 (1%)]	Loss: 27.663984
Train Epoch: 13 [36000/39785 (1%)]	Loss: 23.597200
Train Epoch: 13 [37000/39785 (1%)]	Loss: 26.991192
Train Epoch: 13 [38000/39785 (1%)]	Loss: 24.231863
Train Epoch: 13 [39000/39785 (1%)]	Loss: 24.855963
Train Epoch: 14 [1000/39785 (0%)]	Loss: 23.244192
Train Epoch: 14 [2000/39785 (0%)]	Loss: 26.827066
Train Epoch: 14 [3000/39785 (0%)]	Loss: 26.714289
Train Epoch: 14 [4000/39785 (0%)]	Loss: 26.742910
Train Epoch: 14 [5000/39785 (0%)]	L

Train Epoch: 17 [31000/39785 (1%)]	Loss: 28.341581
Train Epoch: 17 [32000/39785 (1%)]	Loss: 22.179310
Train Epoch: 17 [33000/39785 (1%)]	Loss: 24.968006
Train Epoch: 17 [34000/39785 (1%)]	Loss: 25.764803
Train Epoch: 17 [35000/39785 (1%)]	Loss: 27.684168
Train Epoch: 17 [36000/39785 (1%)]	Loss: 25.411375
Train Epoch: 17 [37000/39785 (1%)]	Loss: 28.350323
Train Epoch: 17 [38000/39785 (1%)]	Loss: 31.054224
Train Epoch: 17 [39000/39785 (1%)]	Loss: 20.608822
Train Epoch: 18 [1000/39785 (0%)]	Loss: 22.640676
Train Epoch: 18 [2000/39785 (0%)]	Loss: 28.908781
Train Epoch: 18 [3000/39785 (0%)]	Loss: 22.096617
Train Epoch: 18 [4000/39785 (0%)]	Loss: 26.823452
Train Epoch: 18 [5000/39785 (0%)]	Loss: 25.277042
Train Epoch: 18 [6000/39785 (0%)]	Loss: 24.861174
Train Epoch: 18 [7000/39785 (0%)]	Loss: 25.793291
Train Epoch: 18 [8000/39785 (0%)]	Loss: 28.109453
Train Epoch: 18 [9000/39785 (0%)]	Loss: 24.596329
Train Epoch: 18 [10000/39785 (0%)]	Loss: 25.332283
Train Epoch: 18 [11000/39785 (0%)]	Loss:

Train Epoch: 22 [37000/39785 (1%)]	Loss: 26.466637
Train Epoch: 22 [38000/39785 (1%)]	Loss: 24.819136
Train Epoch: 22 [39000/39785 (1%)]	Loss: 28.324224
Train Epoch: 23 [1000/39785 (0%)]	Loss: 28.958685
Train Epoch: 23 [2000/39785 (0%)]	Loss: 27.610186
Train Epoch: 23 [3000/39785 (0%)]	Loss: 27.235241
Train Epoch: 23 [4000/39785 (0%)]	Loss: 24.641510
Train Epoch: 23 [5000/39785 (0%)]	Loss: 23.856258
Train Epoch: 23 [6000/39785 (0%)]	Loss: 23.594107
Train Epoch: 23 [7000/39785 (0%)]	Loss: 23.593330
Train Epoch: 23 [8000/39785 (0%)]	Loss: 27.665970
Train Epoch: 23 [9000/39785 (0%)]	Loss: 27.895754
Train Epoch: 23 [10000/39785 (0%)]	Loss: 33.166286
Train Epoch: 23 [11000/39785 (0%)]	Loss: 28.109337
Train Epoch: 23 [12000/39785 (0%)]	Loss: 25.986193
Train Epoch: 23 [13000/39785 (0%)]	Loss: 25.531797
Train Epoch: 23 [14000/39785 (0%)]	Loss: 25.783121
Train Epoch: 23 [15000/39785 (0%)]	Loss: 24.092127
Train Epoch: 23 [16000/39785 (0%)]	Loss: 26.370464
Train Epoch: 23 [17000/39785 (0%)]	Loss:

KeyboardInterrupt: 

In [256]:
answer_test = model_custom_loss(torch.from_numpy(test_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [257]:
for i in range(10):
    print(answer_test[i][:NUM_CLASSES])
    print(torch.argmax(answer_test[i][:NUM_CLASSES]), answer_test[i][NUM_CLASSES:].data.tolist())
    print(test_set._labels[i])
    print()

tensor(8) [0.878501296043396, 0.14600880444049835, -0.009620994329452515, -0.04242924600839615, -0.10463349521160126, 0.5524479150772095]
[ 7.          0.8140826   0.16944312  0.01647427 -0.07471883 -0.16377677
  0.5252441 ]

tensor(8) [0.014157235622406006, 0.8191781044006348, 0.25201231241226196, -0.5993931889533997, 0.038873735815286636, -0.15627287328243256]
[ 7.          0.01401387  0.8029993   0.18298681 -0.5421038   0.071615
 -0.15002087]

tensor(3) [0.929155707359314, 0.052798449993133545, 0.00979757308959961, -0.04924049228429794, -0.08797045052051544, -0.28930455446243286]
[ 7.0000000e+00  9.5146030e-01  4.8440255e-02  9.9439159e-05
 -3.1038227e-03  1.3755901e-02 -3.0360824e-01]

tensor(9) [0.13180002570152283, 0.5320850610733032, 0.3538301885128021, -0.5595158934593201, -0.3730732798576355, 0.5256956815719604]
[ 7.          0.23470888  0.4653611   0.29993004 -0.5283479  -0.37522325
  0.46738502]

tensor(7) [0.5261337757110596, 0.3531031310558319, 0.18525788187980652, -0.3541

---

In [258]:
def weighted_custom_loss(output, target):
    order_out = output[:, 0 : NUM_CLASSES]
    order_target = target[:, 0 : 1].type(torch.LongTensor).squeeze_()
    axis_out = output[:, NUM_CLASSES : NUM_CLASSES + 6]
    axis_target = target[:, 1 : 7]

    loss = 1 * nn.CrossEntropyLoss()(order_out, order_target) + nn.MSELoss(reduction='sum')(axis_out, axis_target)
    
    return loss

In [265]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SE3Convolution(repr_in_1, repr_out_1, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.lin2 = nn.Linear(n_output_1, NUM_CLASSES+6)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) 
        x = F.leaky_relu(self.lin1(x))
        return self.lin2(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = custom_loss(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = weighted_custom_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) 
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [266]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2),(2,3),(2,4),(2,5)]
repr_in_2 = [(2,0),(2,1),(2,2),(2,3),(2,4),(2,5)]
repr_out_2 = [(1,0),(2,1)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 189
n_output_1 = 74 
n_output_2 = 40

batch_size = 100
prob = 0.5
NUM_CLASSES = 10

In [267]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [268]:
epochs = 100
device = torch.device('cpu')
torch.manual_seed(1)

model_fifth_order = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_fifth_order.parameters(), lr=learning_rate)

per_epoch = 10
decr_rate = 0.99

for epoch in range(1, epochs + 1):
    train(model_fifth_order, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

compute 16.pkl.gz... save 16.pkl.gz... done
compute 5.pkl.gz... save 5.pkl.gz... done
compute 17.pkl.gz... save 17.pkl.gz... done
compute 18.pkl.gz... save 18.pkl.gz... done
compute 19.pkl.gz... save 19.pkl.gz... done
compute 20.pkl.gz... save 20.pkl.gz... done
compute 21.pkl.gz... save 21.pkl.gz... done
compute 22.pkl.gz... save 22.pkl.gz... done
compute 23.pkl.gz... save 23.pkl.gz... done
compute 24.pkl.gz... save 24.pkl.gz... done
compute 6.pkl.gz... save 6.pkl.gz... done
compute 25.pkl.gz... save 25.pkl.gz... done
Train Epoch: 1 [1000/39785 (0%)]	Loss: 40.160805
Train Epoch: 1 [2000/39785 (0%)]	Loss: 40.602524
Train Epoch: 1 [3000/39785 (0%)]	Loss: 32.945190
Train Epoch: 1 [4000/39785 (0%)]	Loss: 29.836855
Train Epoch: 1 [5000/39785 (0%)]	Loss: 24.029175
Train Epoch: 1 [6000/39785 (0%)]	Loss: 30.144970
Train Epoch: 1 [7000/39785 (0%)]	Loss: 26.246704
Train Epoch: 1 [8000/39785 (0%)]	Loss: 17.176540
Train Epoch: 1 [9000/39785 (0%)]	Loss: 21.882401
Train Epoch: 1 [10000/39785 (0%)]	L

Train Epoch: 4 [39000/39785 (1%)]	Loss: 15.258318
Train Epoch: 5 [1000/39785 (0%)]	Loss: 11.483993
Train Epoch: 5 [2000/39785 (0%)]	Loss: 11.006619
Train Epoch: 5 [3000/39785 (0%)]	Loss: 14.269030
Train Epoch: 5 [4000/39785 (0%)]	Loss: 11.494339
Train Epoch: 5 [5000/39785 (0%)]	Loss: 13.041487
Train Epoch: 5 [6000/39785 (0%)]	Loss: 13.576275
Train Epoch: 5 [7000/39785 (0%)]	Loss: 16.329958
Train Epoch: 5 [8000/39785 (0%)]	Loss: 11.608188
Train Epoch: 5 [9000/39785 (0%)]	Loss: 13.923944
Train Epoch: 5 [10000/39785 (0%)]	Loss: 12.059420
Train Epoch: 5 [11000/39785 (0%)]	Loss: 13.456617
Train Epoch: 5 [12000/39785 (0%)]	Loss: 10.243171
Train Epoch: 5 [13000/39785 (0%)]	Loss: 16.861450
Train Epoch: 5 [14000/39785 (0%)]	Loss: 15.189270
Train Epoch: 5 [15000/39785 (0%)]	Loss: 12.037784
Train Epoch: 5 [16000/39785 (0%)]	Loss: 14.140921
Train Epoch: 5 [17000/39785 (0%)]	Loss: 16.919897
Train Epoch: 5 [18000/39785 (0%)]	Loss: 15.057847
Train Epoch: 5 [19000/39785 (0%)]	Loss: 15.233223
Train Epo

KeyboardInterrupt: 

---

In [3]:
def weighted_custom_loss(output, target):
    order_out = output[:, 0 : NUM_CLASSES]
    order_target = target[:, 0 : 1].type(torch.LongTensor).squeeze_()
    axis_out = output[:, NUM_CLASSES : NUM_CLASSES + 6]
    axis_target = target[:, 1 : 7]

    loss = 1 * nn.CrossEntropyLoss()(order_out, order_target) + nn.MSELoss(reduction='sum')(axis_out, axis_target)
    
    return loss

In [4]:
class EquiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SE3Convolution(repr_in_1, repr_out_1, size=4)
        self.pool1 = nn.AvgPool3d(pool_size, pool_stride)
        self.conv2 = SE3Convolution(repr_in_2, repr_out_2, size=4)
        self.pool2 = nn.AvgPool3d(pool_size, pool_stride)
        
        self.lin1 = nn.Linear(n_input_1, n_output_1)
        self.lin2 = nn.Linear(n_output_1, NUM_CLASSES+6)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(batch_size,-1) 
        x = F.leaky_relu(self.lin1(x))
        return self.lin2(x)
    
    
def train(model, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate):
    model.train()
    flag = True
    new_epoch = True
    
    if new_epoch:
        batch_idx = 1
        new_epoch = False
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = weighted_custom_loss(output, target)
        loss.backward()
        optimizer.step()
        
    while (train_set._index_in_epoch + batch_size) < train_set._num_examples:
        batch_idx += 1
        data, target, _ = train_set.next_batch(batch_size)
        data = torch.from_numpy(data.reshape(batch_size,1,24,24,24)).type(torch.FloatTensor)
        target = torch.from_numpy(target.reshape(batch_size,-1)).type(torch.FloatTensor)
        cnt = epoch // per_epoch
        if ((epoch+1) // per_epoch > cnt) and flag:
            lr = 0
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            lr *= decr_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            flag = False
        if ((epoch+1) // per_epoch <= cnt):
            flag = True
            
        optimizer.zero_grad()
        output = model(data)
        # loss_fn = nn.MSELoss(reduction='sum')
        # loss = loss_fn(output, target)
        loss = weighted_custom_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, train_set._num_examples,
                100. * batch_idx / train_set._num_examples, loss.item()))

def test(model, device, test_set):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_set:
            output = model(data)
            loss_fn = nn.MSELoss(reduction='sum')
            test_loss += loss_fn(output, target) 
            # pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += int(torch.argmax(output) == torch.argmax(target))

    test_loss /= len(test_set)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_set),
        100. * correct / len(test_set)))
    return test_loss

In [11]:
repr_in_1 = [(1,0)]
repr_out_1 = [(2,0),(2,1),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9)]
repr_in_2 = [(2,0),(2,1),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9)]
repr_out_2 = [(1,0),(2,1),(2,2),(2,3)]
size = 4
activation = (None, F.leaky_relu)
pool_size = 2
pool_stride = 2
bias = True

n_input_1 = 837
n_output_1 = 420 
n_output_2 = 40

batch_size = 100
prob = 0.5
NUM_CLASSES = 10

In [7]:
train_name = '../deepSymmetry/data/fromScripts/dataReady0'
train_set = load_data.read_data_set(train_name, dtype=dtypes.float16, seed = 1)

Extracting ../deepSymmetry/data/fromScripts/dataReady0
Extracting ../deepSymmetry/data/fromScripts/dataReady0_label
(39785, 13824)


In [ ]:
epochs = 100
device = torch.device('cpu')
torch.manual_seed(1)

model_tenth_order = EquiNet().to(device)
learning_rate = 5e-3
optimizer = torch.optim.Adam(model_tenth_order.parameters(), lr=learning_rate)

per_epoch = 10
decr_rate = 0.995

for epoch in range(1, epochs + 1):
    train(model_tenth_order, device, train_set, batch_size, optimizer, epoch, per_epoch, decr_rate)
    # test(model_hard, device, test_set)

Train Epoch: 1 [1000/39785 (0%)]	Loss: 36.971638
Train Epoch: 1 [2000/39785 (0%)]	Loss: 27.032562
Train Epoch: 1 [3000/39785 (0%)]	Loss: 23.561869
Train Epoch: 1 [4000/39785 (0%)]	Loss: 24.894772
Train Epoch: 1 [5000/39785 (0%)]	Loss: 20.401770
Train Epoch: 1 [6000/39785 (0%)]	Loss: 24.369877
Train Epoch: 1 [7000/39785 (0%)]	Loss: 21.327436
Train Epoch: 1 [8000/39785 (0%)]	Loss: 18.343349
Train Epoch: 1 [9000/39785 (0%)]	Loss: 21.651529
Train Epoch: 1 [10000/39785 (0%)]	Loss: 20.213943
Train Epoch: 1 [11000/39785 (0%)]	Loss: 19.364433
Train Epoch: 1 [12000/39785 (0%)]	Loss: 18.184734
Train Epoch: 1 [13000/39785 (0%)]	Loss: 19.650126
Train Epoch: 1 [14000/39785 (0%)]	Loss: 19.353704
Train Epoch: 1 [15000/39785 (0%)]	Loss: 19.617815
Train Epoch: 1 [16000/39785 (0%)]	Loss: 18.656876
Train Epoch: 1 [17000/39785 (0%)]	Loss: 18.036095
Train Epoch: 1 [18000/39785 (0%)]	Loss: 15.091098
Train Epoch: 1 [19000/39785 (0%)]	Loss: 21.126528
Train Epoch: 1 [20000/39785 (1%)]	Loss: 18.754930
Train Epo

Train Epoch: 5 [10000/39785 (0%)]	Loss: 10.510989
Train Epoch: 5 [11000/39785 (0%)]	Loss: 10.718228
Train Epoch: 5 [12000/39785 (0%)]	Loss: 7.922806
Train Epoch: 5 [13000/39785 (0%)]	Loss: 14.894974
Train Epoch: 5 [14000/39785 (0%)]	Loss: 11.784818
Train Epoch: 5 [15000/39785 (0%)]	Loss: 9.821644
Train Epoch: 5 [16000/39785 (0%)]	Loss: 10.861672
Train Epoch: 5 [17000/39785 (0%)]	Loss: 14.176741
Train Epoch: 5 [18000/39785 (0%)]	Loss: 12.759933
Train Epoch: 5 [19000/39785 (0%)]	Loss: 12.121784
Train Epoch: 5 [20000/39785 (1%)]	Loss: 9.804792
Train Epoch: 5 [21000/39785 (1%)]	Loss: 11.179785
Train Epoch: 5 [22000/39785 (1%)]	Loss: 10.303892
Train Epoch: 5 [23000/39785 (1%)]	Loss: 11.660330
Train Epoch: 5 [24000/39785 (1%)]	Loss: 11.954089
Train Epoch: 5 [25000/39785 (1%)]	Loss: 11.476882
Train Epoch: 5 [26000/39785 (1%)]	Loss: 10.912248
Train Epoch: 5 [27000/39785 (1%)]	Loss: 8.913859
Train Epoch: 5 [28000/39785 (1%)]	Loss: 12.401588
Train Epoch: 5 [29000/39785 (1%)]	Loss: 11.974815
Trai

Train Epoch: 9 [20000/39785 (1%)]	Loss: 9.403354
Train Epoch: 9 [21000/39785 (1%)]	Loss: 7.567673
Train Epoch: 9 [22000/39785 (1%)]	Loss: 7.521990
Train Epoch: 9 [23000/39785 (1%)]	Loss: 10.357711
Train Epoch: 9 [24000/39785 (1%)]	Loss: 10.111736
Train Epoch: 9 [25000/39785 (1%)]	Loss: 11.512489
Train Epoch: 9 [26000/39785 (1%)]	Loss: 10.731125
Train Epoch: 9 [27000/39785 (1%)]	Loss: 10.346197
Train Epoch: 9 [28000/39785 (1%)]	Loss: 12.059620
Train Epoch: 9 [29000/39785 (1%)]	Loss: 10.319165
Train Epoch: 9 [30000/39785 (1%)]	Loss: 9.478619
Train Epoch: 9 [31000/39785 (1%)]	Loss: 10.168149
Train Epoch: 9 [32000/39785 (1%)]	Loss: 8.625074
Train Epoch: 9 [33000/39785 (1%)]	Loss: 11.278180
Train Epoch: 9 [34000/39785 (1%)]	Loss: 10.272439
Train Epoch: 9 [35000/39785 (1%)]	Loss: 6.097641
Train Epoch: 9 [36000/39785 (1%)]	Loss: 10.355350
Train Epoch: 9 [37000/39785 (1%)]	Loss: 10.753444
Train Epoch: 9 [38000/39785 (1%)]	Loss: 11.263067
Train Epoch: 9 [39000/39785 (1%)]	Loss: 10.748221
Train 

Train Epoch: 13 [29000/39785 (1%)]	Loss: 6.999557
Train Epoch: 13 [30000/39785 (1%)]	Loss: 8.564786
Train Epoch: 13 [31000/39785 (1%)]	Loss: 8.425055
Train Epoch: 13 [32000/39785 (1%)]	Loss: 5.095156
Train Epoch: 13 [33000/39785 (1%)]	Loss: 6.971272
Train Epoch: 13 [34000/39785 (1%)]	Loss: 9.011045
Train Epoch: 13 [35000/39785 (1%)]	Loss: 9.061323
Train Epoch: 13 [36000/39785 (1%)]	Loss: 6.386374
Train Epoch: 13 [37000/39785 (1%)]	Loss: 7.678381
Train Epoch: 13 [38000/39785 (1%)]	Loss: 7.761443
Train Epoch: 13 [39000/39785 (1%)]	Loss: 7.863189
Train Epoch: 14 [1000/39785 (0%)]	Loss: 6.257151
Train Epoch: 14 [2000/39785 (0%)]	Loss: 6.757326
Train Epoch: 14 [3000/39785 (0%)]	Loss: 7.111325
Train Epoch: 14 [4000/39785 (0%)]	Loss: 6.957067
Train Epoch: 14 [5000/39785 (0%)]	Loss: 6.344107
Train Epoch: 14 [6000/39785 (0%)]	Loss: 6.484881
Train Epoch: 14 [7000/39785 (0%)]	Loss: 6.959034
Train Epoch: 14 [8000/39785 (0%)]	Loss: 6.560190
Train Epoch: 14 [9000/39785 (0%)]	Loss: 7.983716
Train Epo

Train Epoch: 17 [38000/39785 (1%)]	Loss: 9.349587
Train Epoch: 17 [39000/39785 (1%)]	Loss: 5.430182
Train Epoch: 18 [1000/39785 (0%)]	Loss: 5.089432
Train Epoch: 18 [2000/39785 (0%)]	Loss: 5.682003
Train Epoch: 18 [3000/39785 (0%)]	Loss: 5.950202
Train Epoch: 18 [4000/39785 (0%)]	Loss: 5.828528
Train Epoch: 18 [5000/39785 (0%)]	Loss: 7.059165
Train Epoch: 18 [6000/39785 (0%)]	Loss: 6.049141
Train Epoch: 18 [7000/39785 (0%)]	Loss: 5.357966
Train Epoch: 18 [8000/39785 (0%)]	Loss: 5.727799
Train Epoch: 18 [9000/39785 (0%)]	Loss: 6.365763
Train Epoch: 18 [10000/39785 (0%)]	Loss: 5.479970
Train Epoch: 18 [11000/39785 (0%)]	Loss: 5.783562
Train Epoch: 18 [12000/39785 (0%)]	Loss: 6.516585
Train Epoch: 18 [13000/39785 (0%)]	Loss: 5.314655
Train Epoch: 18 [14000/39785 (0%)]	Loss: 6.594130
Train Epoch: 18 [15000/39785 (0%)]	Loss: 5.946997
Train Epoch: 18 [16000/39785 (0%)]	Loss: 5.620379
Train Epoch: 18 [17000/39785 (0%)]	Loss: 7.400437
Train Epoch: 18 [18000/39785 (0%)]	Loss: 8.443272
Train Epo

Train Epoch: 23 [8000/39785 (0%)]	Loss: 7.045302
Train Epoch: 23 [9000/39785 (0%)]	Loss: 5.849342
Train Epoch: 23 [10000/39785 (0%)]	Loss: 8.144588
Train Epoch: 23 [11000/39785 (0%)]	Loss: 5.854420
Train Epoch: 23 [12000/39785 (0%)]	Loss: 5.645767
Train Epoch: 23 [13000/39785 (0%)]	Loss: 4.330109
Train Epoch: 23 [14000/39785 (0%)]	Loss: 4.016644
Train Epoch: 23 [15000/39785 (0%)]	Loss: 5.574136
Train Epoch: 23 [16000/39785 (0%)]	Loss: 4.556806
Train Epoch: 23 [17000/39785 (0%)]	Loss: 4.918741
Train Epoch: 23 [18000/39785 (0%)]	Loss: 4.217258
Train Epoch: 23 [19000/39785 (0%)]	Loss: 4.552353
Train Epoch: 23 [20000/39785 (1%)]	Loss: 4.593202
Train Epoch: 23 [21000/39785 (1%)]	Loss: 6.643904
Train Epoch: 23 [22000/39785 (1%)]	Loss: 4.639333
Train Epoch: 23 [23000/39785 (1%)]	Loss: 4.600451
Train Epoch: 23 [24000/39785 (1%)]	Loss: 6.506300
Train Epoch: 23 [25000/39785 (1%)]	Loss: 6.356036
Train Epoch: 23 [26000/39785 (1%)]	Loss: 7.719636
Train Epoch: 23 [27000/39785 (1%)]	Loss: 6.775963
Tr

Train Epoch: 27 [17000/39785 (0%)]	Loss: 6.703030
Train Epoch: 27 [18000/39785 (0%)]	Loss: 4.834484
Train Epoch: 27 [19000/39785 (0%)]	Loss: 5.962868
Train Epoch: 27 [20000/39785 (1%)]	Loss: 4.395610
Train Epoch: 27 [21000/39785 (1%)]	Loss: 4.678164
Train Epoch: 27 [22000/39785 (1%)]	Loss: 4.536694
Train Epoch: 27 [23000/39785 (1%)]	Loss: 5.460795
Train Epoch: 27 [24000/39785 (1%)]	Loss: 4.813757
Train Epoch: 27 [25000/39785 (1%)]	Loss: 5.027262
Train Epoch: 27 [26000/39785 (1%)]	Loss: 4.893330
Train Epoch: 27 [27000/39785 (1%)]	Loss: 4.772455
Train Epoch: 27 [28000/39785 (1%)]	Loss: 4.294953
Train Epoch: 27 [29000/39785 (1%)]	Loss: 5.553070
Train Epoch: 27 [30000/39785 (1%)]	Loss: 5.501843
Train Epoch: 27 [31000/39785 (1%)]	Loss: 4.970952
Train Epoch: 27 [32000/39785 (1%)]	Loss: 4.399627
Train Epoch: 27 [33000/39785 (1%)]	Loss: 6.837511
Train Epoch: 27 [34000/39785 (1%)]	Loss: 5.460639
Train Epoch: 27 [35000/39785 (1%)]	Loss: 5.336627
Train Epoch: 27 [36000/39785 (1%)]	Loss: 4.230852


Train Epoch: 31 [26000/39785 (1%)]	Loss: 4.051652
Train Epoch: 31 [27000/39785 (1%)]	Loss: 3.571855
Train Epoch: 31 [28000/39785 (1%)]	Loss: 3.602245
Train Epoch: 31 [29000/39785 (1%)]	Loss: 5.301577
Train Epoch: 31 [30000/39785 (1%)]	Loss: 5.108009
Train Epoch: 31 [31000/39785 (1%)]	Loss: 6.239379
Train Epoch: 31 [32000/39785 (1%)]	Loss: 4.565140
Train Epoch: 31 [33000/39785 (1%)]	Loss: 4.261871
Train Epoch: 31 [34000/39785 (1%)]	Loss: 3.754645
Train Epoch: 31 [35000/39785 (1%)]	Loss: 4.443115
Train Epoch: 31 [36000/39785 (1%)]	Loss: 5.882146
Train Epoch: 31 [37000/39785 (1%)]	Loss: 6.626326
Train Epoch: 31 [38000/39785 (1%)]	Loss: 4.204153
Train Epoch: 31 [39000/39785 (1%)]	Loss: 4.405591
Train Epoch: 32 [1000/39785 (0%)]	Loss: 4.633092
Train Epoch: 32 [2000/39785 (0%)]	Loss: 3.043011
Train Epoch: 32 [3000/39785 (0%)]	Loss: 5.957927
Train Epoch: 32 [4000/39785 (0%)]	Loss: 3.669952
Train Epoch: 32 [5000/39785 (0%)]	Loss: 3.747842
Train Epoch: 32 [6000/39785 (0%)]	Loss: 3.779496
Train 

Train Epoch: 35 [35000/39785 (1%)]	Loss: 4.379299
Train Epoch: 35 [36000/39785 (1%)]	Loss: 3.716275
Train Epoch: 35 [37000/39785 (1%)]	Loss: 4.543982
Train Epoch: 35 [38000/39785 (1%)]	Loss: 4.471198
Train Epoch: 35 [39000/39785 (1%)]	Loss: 4.058135
Train Epoch: 36 [1000/39785 (0%)]	Loss: 3.643576
Train Epoch: 36 [2000/39785 (0%)]	Loss: 3.249705
Train Epoch: 36 [3000/39785 (0%)]	Loss: 4.175546
Train Epoch: 36 [4000/39785 (0%)]	Loss: 3.697476
Train Epoch: 36 [5000/39785 (0%)]	Loss: 3.509823
Train Epoch: 36 [6000/39785 (0%)]	Loss: 3.326844
Train Epoch: 36 [7000/39785 (0%)]	Loss: 3.856164
Train Epoch: 36 [8000/39785 (0%)]	Loss: 4.578734
Train Epoch: 36 [9000/39785 (0%)]	Loss: 4.721083
Train Epoch: 36 [10000/39785 (0%)]	Loss: 3.614054
Train Epoch: 36 [11000/39785 (0%)]	Loss: 3.959548
Train Epoch: 36 [12000/39785 (0%)]	Loss: 5.391181
Train Epoch: 36 [13000/39785 (0%)]	Loss: 4.949510
Train Epoch: 36 [14000/39785 (0%)]	Loss: 4.290684
Train Epoch: 36 [15000/39785 (0%)]	Loss: 5.702629
Train Epo

In [ ]:
answer_test = model_tenth_order(torch.from_numpy(test_set._images[:100].reshape(batch_size,1,24,24,24)).type(torch.FloatTensor))

In [ ]:
for i in range(10):
    print(answer_test[i][:NUM_CLASSES])
    print(torch.argmax(answer_test[i][:NUM_CLASSES]), answer_test[i][NUM_CLASSES:].data.tolist())
    print(test_set._labels[i])
    print()